# Check sklearn RBF implement

In [ ]:
from scipy.spatial.distance import pdist, cdist, squareform

x = np.array([[30, 1, 2500],
              [70, 0, 2000],
              [50, 1, 3000]])
y = x
l = np.array([50, 1, 240])
# RBF
dists = cdist(x/l, y/l, metric="sqeuclidean")
print(dists)
# ARD
VI = np.zeros((x.shape[1], x.shape[1]))
np.fill_diagonal(VI, 1/(l**2))
np.dot(np.dot((x-y), VI),(x-y).T)
print(dists)

# Check model
## Model was generated in normative.py

In [ ]:
import os
import pickle
save_dir = './results_0401/normative_model3/combat_gmv/age_gender_tiv'

with open(os.path.join(save_dir, f'model/gpr_0.pkl'), 'rb') as f:
    gaussian_process = pickle.load(f)
params = gaussian_process.kernel_.get_params()
for key in sorted(params): print("%s : %s" % (key, params[key]))

In [ ]:
import os
import pickle
save_dir = './results_0401/normative_model3/combat_gmv/age_gender_tiv'

models = []
for roi in range(246):
    with open(os.path.join(save_dir, f'model/gpr_{roi}.pkl'), 'rb') as f:
        gaussian_process = pickle.load(f)
    models.append(gaussian_process)

In [ ]:
for model in models:
    print(model.kernel_)

# Load Data

In [ ]:
import os
from matplotlib import markers
from sklearn.metrics import zero_one_loss

from sklearn.model_selection import KFold
import datasets
import numpy as np
import pandas as pd

# Construct Normative model (GPR) using NC data
# x: age, gender | y: ROI_i
# Loading NC 

def prepare_data(centers, prefix, target_label):
    center_names = []
    persons = []
    MMSEs = []
    ages = []
    genders = []
    tivs = []
    acts = []
    all_features = None
    for center in centers:
        persons_ = center.get_by_label(target_label)
        if persons_:
            center_names += [center.name for person in persons_]
            persons += [person.filename for person in persons_]
            MMSEs += center.get_MMSEs(target_label)[0].tolist()
            ages += center.get_ages(target_label)[0].tolist()
            genders += center.get_males(target_label)[0].tolist()
            tivs += center.get_tivs(target_label)[0].tolist()
            acts += center.get_average_thickness(target_label)[0].tolist()

            features, *_ = center.get_csv_values(persons=persons_, prefix=prefix, flatten=True)
            if all_features is None:
                all_features = features
            else:
                all_features = np.vstack((all_features, features))
    return center_names, persons, MMSEs, ages, genders, tivs, acts, all_features

In [ ]:
import pickle
import threading

centers = datasets.load_centers_all()

prefix = 'neurocombat_ct2/{}.csv'
train_label = 0
center_names, persons, MMSEs, ages, genders, tivs, acts, all_features = prepare_data(centers, prefix, train_label)

mci_label = 1
center_names1, persons1, MMSEs1, ages1, genders1, tivs1, acts1, all_features1 = prepare_data(centers, prefix, mci_label)

ad_label = 2
center_names2, persons2, MMSEs2, ages2, genders2, tivs2, acts2, all_features2 = prepare_data(centers, prefix, ad_label)

## 2D plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

model_dir = './results_0401/normative_model3/combat_ct/age_gender/model'

def prepare_xy(roi, ages, genders, all_features, tivs, acts):
    x = []
    y = []
    for age, gender, rois, tiv, act in zip(ages, genders, all_features, tivs, acts):
        # gmv: tiv/1000, CT: None
        value = rois[roi]
        x.append([age, gender])
        y.append([value])
    x = np.array(x)
    y = np.array(y)
    return x, y

roi = 192
x, y = prepare_xy(roi, ages, genders, all_features, tivs, acts)

x1, y1 = prepare_xy(roi, ages1, genders1, all_features1, tivs1, acts1)
x2, y2 = prepare_xy(roi, ages2, genders2, all_features2, tivs2, acts2)

with open(os.path.join(model_dir, f'gpr_{roi}.pkl'), 'rb') as ff:
    gaussian_process = pickle.load(ff)

arr1inds = x[:, 0].argsort()
x_sorted = x[arr1inds[::-1]]
mean_prediction, std_prediction = gaussian_process.predict(x_sorted, return_std=True)

s = 5
plt.scatter(x[:, 0], y, label=r"NC", c='#00865c', s=s)
plt.scatter(x1[:, 0], y1, label=r"MCI", c='#aa3424', s=s, marker='s')
plt.scatter(x2[:, 0], y2, label=r"AD", c='#173172', s=s, marker='*')
plt.scatter(x_sorted[:, 0], mean_prediction, label="Mean prediction")
plt.fill_between(
    x_sorted[:, 0].ravel(),
    mean_prediction.ravel() - 1.96 * std_prediction,
    mean_prediction.ravel() + 1.96 * std_prediction,
    alpha=0.5,
    label=r"95% confidence interval",
)

plt.show()

## 3D Plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib widget

model_dir = './results_0401/normative_model3/combat_gmv/age_gender_tiv/model'

def prepare_xy(roi, ages, genders, all_features, tivs, acts):
    x = []
    y = []
    for age, gender, rois, tiv, act in zip(ages, genders, all_features, tivs, acts):
        # gmv: tiv/1000, CT: None
        value = rois[roi]
        if tiv > 800:
            x.append([age, gender, tiv/1000])
            y.append([value])
    x = np.array(x)
    y = np.array(y)
    return x, y

roi = 232
x, y = prepare_xy(roi, ages, genders, all_features, tivs, acts)

x1, y1 = prepare_xy(roi, ages1, genders1, all_features1, tivs1, acts1)
x2, y2 = prepare_xy(roi, ages2, genders2, all_features2, tivs2, acts2)

fig = plt.figure(figsize=(10, 10))

with open(os.path.join(model_dir, f'gpr_{roi}.pkl'), 'rb') as ff:
    gaussian_process = pickle.load(ff)

mean_prediction, std_prediction = gaussian_process.predict(x, return_std=True)
mean_prediction = mean_prediction.ravel()
std_prediction = std_prediction.ravel()

ax = fig.add_subplot(1, 1, 1, projection='3d')

a = x[:, 0]
b = x[:, 2]

a1 = x1[:, 0]
b1 = x1[:, 2]

a2 = x2[:, 0]
b2 = x2[:, 2]

shape = a.shape
#a, b = np.meshgrid(a, b, mean_prediction.ravel(), std_prediction)
s = 5
ax.scatter(a, b, y, marker='.', s=s, label='NC', color='#00865c')
ax.scatter(a1, b1, y1, marker='s', s=s, label='MCI', color='#aa3424')
ax.scatter(a2, b2, y2, marker='*', s=s, label='AD', color='#173172')
"""
ax.scatter(a, b, mean_prediction.ravel(), s=5, marker='s', label='Predict Mean')
ax.scatter(a, b, mean_prediction.ravel()+1.96*std_prediction, s=5, marker='^', label='Upper 95%')
ax.scatter(a, b, mean_prediction.ravel()-1.96*std_prediction, s=5, marker='v', label='lower 95%')
"""
surf = ax.plot_trisurf(a, b, mean_prediction, label='Predict Mean', color='#d8ae80',shade=False)
#surf = ax.plot_trisurf(a, b, mean_prediction, label='Predict Mean', shade=False)
surf._facecolors2d = surf._facecolor3d
surf._edgecolors2d = surf._edgecolor3d
"""
surf = ax.plot_trisurf(a, b, mean_prediction+1.96*std_prediction,label='Upper 95%', shade=False)
surf._facecolors2d = surf._facecolor3d
surf._edgecolors2d = surf._edgecolor3d
surf = ax.plot_trisurf(a, b, mean_prediction-1.96*std_prediction,label='lower 95%', shade=False)
surf._facecolors2d = surf._facecolor3d
surf._edgecolors2d = surf._edgecolor3d
"""
ax.set_xlabel("$Age$")
ax.set_ylabel("$TIV$")
ax.set_zlabel("$GMV$")
#ax.set_zlim((500,3500))
ax.legend()

In [ ]:
x

In [ ]:
np.meshgrid(mean_prediction.ravel())[0].shape

## 3D plot for gender

In [ ]:
%matplotlib widget

fig = plt.figure(figsize=(10, 10))

ax = fig.add_subplot(1, 1, 1, projection='3d')

a = x_sorted[x_sorted[:, 1]==0][:, 0]
b = x_sorted[x_sorted[:, 1]==0][:, 1]
y1 = y[x_sorted[:, 1]==0]
mean_prediction1 = mean_prediction[x_sorted[:, 1]==0]
std_prediction1 = std_prediction[x_sorted[:, 1]==0]

ax.scatter(a, b, y1, marker='.', s=5, label='True')
surf = ax.scatter(a, b, mean_prediction1.ravel(), label='Predict Mean')

surf = ax.scatter(a, b, mean_prediction1.ravel()+1.96*std_prediction1,label='Upper 95%')

surf = ax.scatter(a, b, mean_prediction1.ravel()-1.96*std_prediction1,label='lower 95%')


c = x_sorted[x_sorted[:, 1]==1][:, 0]
d = x_sorted[x_sorted[:, 1]==1][:, 1]
y2 = y[x_sorted[:, 1]==1]
mean_prediction2 = mean_prediction[x_sorted[:, 1]==1]
std_prediction2 = std_prediction[x_sorted[:, 1]==1]

ax.scatter(c, d, y2, marker='.', s=5, label='True')
surf = ax.scatter(c, d, mean_prediction2.ravel(), label='Predict Mean')

surf = ax.scatter(c, d, mean_prediction2.ravel()+1.96*std_prediction2,label='Upper 95%')

surf = ax.scatter(c, d, mean_prediction2.ravel()-1.96*std_prediction2,label='lower 95%')


ax.set_xlabel("$Age$")
ax.set_ylabel("$Gender$")
ax.set_zlabel("$GMV$")
ax.legend()

## Test

In [ ]:
target_label = 2
center_names, persons, MMSEs, ages, genders, tivs, all_features = prepare_data(centers, prefix, target_label)

x_test = []
y_test = []
for age, gender, rois, MMSE, tiv in zip(ages, genders, all_features, MMSEs, tivs):
    x_test.append([age*100, tiv])
    y_test.append(rois[217])
x_test = np.array(x_test)
y_test = np.array(y_test)
mean_prediction, std_prediction = gaussian_process.predict(x_test, return_std=True)

plt.scatter(x[:, 0], y, label=r"Train", s=0.2)
plt.scatter(x_test[:, 0], y_test, label=r"Test", c='#2ca02c', s=0.2)
plt.scatter(x_test[:, 0], mean_prediction, label="Mean prediction", s=5)
plt.legend()
plt.xlabel("$Age$")
plt.ylabel("$Mean GMV Predict$")

z = (y_test - mean_prediction.ravel()) / np.sqrt((std_prediction)**2 + gaussian_process.kernel_.get_params()['k2__noise_level'])
print(z)

## Save scores

In [ ]:
import os
import pandas as pd
import numpy as np
import datasets
import pickle

def cal_personal_scores(centers, model_dir, csv_prefix, out_prefix, gmv=True):
    models = []

    if gmv:
        roi_count = 246
    else:
        roi_count = 210

    for i in range(roi_count):
        with open(os.path.join(model_dir, f'gpr_{i}.pkl'), 'rb') as ff:
            model = pickle.load(ff)
            models.append(model)

    for center in centers:
        file_dir = center.file_dir
        for person in center.persons:
            # Load Features
            csv_path = os.path.join(file_dir,
                                csv_prefix.format(person.filename))
            df = pd.read_csv(csv_path, index_col=0)
            features = df.to_numpy().flatten()
            # Calculate personal scores
            personal_scores = []
            roi_counts = np.shape(features)[0]

            age = center.get_presonal_info_values_by_person(person)[0]
            gender = center.get_presonal_info_values_by_person(person)[1]
            tiv = center.get_tiv_by_person(person)
            act = center.get_average_thickness_by_person(person)

            if gmv:
                x = [[age, gender, tiv/1000]]
            else:
                x = [[age, gender]]

            for i in range(roi_counts):
                y_pred, y_std = models[i].predict(x, return_std=True)

                print()
                personal_score = (features[i] - y_pred.ravel()) / np.sqrt((y_std)**2 + models[i].kernel_.get_params()['k2__noise_level'])[0]
                personal_scores.append(personal_score[0])

            out_path = os.path.join(file_dir,
                                out_prefix.format(person.filename))

            dic = dict(zip(range(1, roi_counts+1), personal_scores))
            df = pd.DataFrame(dic.items(), columns=['roi', 'value'])
            df.set_index('roi', inplace=True)
            df.to_csv(out_path)

In [ ]:
centers = datasets.load_centers_all()
model_dir = './results_0401/normative_model3/combat_gmv/age_gender_tiv/model'
csv_prefix = 'neurocombat_gmv2/{}.csv'
out_prefix = 'ps_g_agt/{}.csv'

cal_personal_scores(centers, model_dir, csv_prefix, out_prefix, gmv=True)

In [ ]:
centers = datasets.load_centers_all()
model_dir = './results_0401/normative_model3/combat_ct/age_gender/model'
csv_prefix = 'neurocombat_ct2/{}.csv'
out_prefix = 'ps_c_ag/{}.csv'

cal_personal_scores(centers, model_dir, csv_prefix, out_prefix, gmv=False)

# Toy Data

In [ ]:
# Toy dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel

X = np.linspace(start=0, stop=10, num=1_000).reshape(-1, 1)
y = np.squeeze(X * np.sin(X))
noise = np.random.normal(-1, 1, 1000)
y = (y + noise) * 100

rng = np.random.RandomState(1)
training_indices = rng.choice(np.arange(y.size), size=500, replace=False)
X_train, y_train = X[training_indices], y[training_indices]

kernel = 1 * RBF(length_scale=[1], length_scale_bounds=(1e-10,1e10)) +\
        WhiteKernel(noise_level_bounds=(1e-5,1e5))
gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=50)
gaussian_process.fit(X_train, y_train)

mean_prediction, std_prediction = gaussian_process.predict(X, return_std=True)

plt.scatter(X, y, label=r"$f(x) = x \sin(x)$", c='#28FF99', s=0.5)
plt.scatter(X_train, y_train, label="Observations", s=5)
plt.plot(X, mean_prediction, label="Mean prediction")
plt.fill_between(
    X.ravel(),
    mean_prediction - 1.96 * std_prediction,
    mean_prediction + 1.96 * std_prediction,
    alpha=0.5,
    label=r"95% confidence interval",
)
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("Gaussian process regression on noise-free dataset")
print(gaussian_process.kernel_)

In [ ]:
gaussian_process.kernel_.get_params()['k2__noise_level']

## Test

In [ ]:
offset = 5

X_test = np.linspace(start=0, stop=10, num=1_000).reshape(-1, 1)
y_test = np.squeeze(X_test * np.sin(X_test) - offset)
noise = np.random.normal(-1, 1, 1000)
y_test = (y_test + noise) * 100

mean_prediction, std_prediction = gaussian_process.predict(X_test, return_std=True)

plt.scatter(X_train, y_train, label=r"$f(x) = x \sin(x)$", c='#28FF99', s=0.5)

plt.scatter(X_test, y_test, label=r"$f(x) = x \sin(x)-{}$".format(offset), c='#FD8D3C', s=0.5)

plt.plot(X_test, mean_prediction, label="Mean prediction")
plt.fill_between(
    X.ravel(),
    mean_prediction - 1.96 * std_prediction,
    mean_prediction + 1.96 * std_prediction,
    alpha=0.5,
    label=r"95% confidence interval",
)
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("Gaussian process regression on noise-free dataset")
plt.show()
plt.close

import seaborn as sns
z = (y_test - mean_prediction) / np.sqrt((std_prediction)**2 + gaussian_process.kernel_.get_params()['k2__noise_level'])
sns.displot(z)